## Try out model from TempoBert
I think the `test_bert` module only works with the 'old' [tempobert](https://github.com/guyrosin/tempobert/) model, where the sentence was prepended with a token. The model [linked on github](https://github.com/guyrosin/temporal_attention/issues/1) is of type "attention_double_time". The tokenizer needs an extra argurment `time_id` which can take 2 values.


In [1]:
from bert_model import BertModel
from train_tempobert import ModelArguments
from transformers import AutoModelForMaskedLM, pipeline

In [2]:
import test_bert
import hf_utils

Load the model:

In [4]:
hf_utils.prepare_tf_classes()
model_name = 'temp_att_model_semeval_eng'
model_args = ModelArguments(model_name_or_path=model_name)
config_kwargs = {}
model, tokenizer = hf_utils.load_pretrained_model(
    model_args,
    AutoModelForMaskedLM,
    expect_times_in_model=True,
    **config_kwargs,
)

In [6]:
tokenizer

PreTrainedTokenizerFast(name_or_path='temp_att_model_semeval_eng', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

We create a pipeline, but automatically calling the pipeline doesn't work because the tokenizer encoding function needs an extra argument `time_id`, so we create a manual function.

In [7]:
fill_mask_pipeline = pipeline("fill-mask", model=model, tokenizer=tokenizer)

In [ ]:
pipeline.

In [37]:
def run_pipeline(pipeline, sentence, time_id):
    encoded_inputs = fill_mask_pipeline.tokenizer.encode_plus(sentence, time_id=time_id, return_tensors='pt')
    output = fill_mask_pipeline.forward(encoded_inputs)
    result = fill_mask_pipeline.postprocess(output)
    return result

In [38]:
sentence = "now any behavior that would earn demerits for a boy scout seems [MASK] game."
run_pipeline(fill_mask_pipeline, sentence, "1")

[{'sequence': 'now any behavior that would earn demerits for a boy scout seems. game.',
  'score': 0.0629839077591896,
  'token': 1012,
  'token_str': '.'},
 {'sequence': 'now any behavior that would earn demerits for a boy scout seems " game.',
  'score': 0.018260018900036812,
  'token': 1000,
  'token_str': '"'},
 {'sequence': 'now any behavior that would earn demerits for a boy scout seems then game.',
  'score': 0.017460325732827187,
  'token': 2059,
  'token_str': 'then'},
 {'sequence': 'now any behavior that would earn demerits for a boy scout seems left game.',
  'score': 0.014366024173796177,
  'token': 2187,
  'token_str': 'left'},
 {'sequence': "now any behavior that would earn demerits for a boy scout seems'game.",
  'score': 0.013853857293725014,
  'token': 1005,
  'token_str': "'"}]

In [39]:
run_pipeline(fill_mask_pipeline, sentence, "2")

[{'sequence': 'now any behavior that would earn demerits for a boy scout seems. game.',
  'score': 0.05730462074279785,
  'token': 1012,
  'token_str': '.'},
 {'sequence': 'now any behavior that would earn demerits for a boy scout seems " game.',
  'score': 0.01933969184756279,
  'token': 1000,
  'token_str': '"'},
 {'sequence': 'now any behavior that would earn demerits for a boy scout seems then game.',
  'score': 0.01873614825308323,
  'token': 2059,
  'token_str': 'then'},
 {'sequence': 'now any behavior that would earn demerits for a boy scout seems left game.',
  'score': 0.01754055730998516,
  'token': 2187,
  'token_str': 'left'},
 {'sequence': "now any behavior that would earn demerits for a boy scout seems'game.",
  'score': 0.015870574861764908,
  'token': 1005,
  'token_str': "'"}]